In [1]:
# importar pacotes necessários
import numpy as np
import pandas as pd

In [2]:
# definir parâmetros extras
pd.set_option('precision', 4)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
input_prefix = 'data/'
#input_prefix = 'https://github.com/hjort/ai-labs/raw/master/jupyter/sales-russia/data/'

bzfile = '.bz2' # ''

## Sales (testing)

In [4]:
df_test = pd.read_csv(
    input_prefix + 'test.csv' + bzfile, #nrows=10000,
    dtype={'shop_id': np.int8, 'item_id': np.int16},
    index_col='ID')
print('shape:', df_test.shape)
df_test.head()

shape: (214200, 2)


,shop_id,item_id
ID,,
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268


In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214200 entries, 0 to 214199
Data columns (total 2 columns):
shop_id    214200 non-null int8
item_id    214200 non-null int16
dtypes: int16(1), int8(1)
memory usage: 2.2 MB


In [6]:
df_test.describe()

,shop_id,item_id
count,214200.0000,214200.0000
mean,31.6429,11019.3986
std,17.5619,6252.6446
min,2.0000,30.0000
25%,16.0000,5381.5000
50%,34.5000,11203.0000
75%,47.0000,16071.5000
max,59.0000,22167.0000


In [7]:
# criar dataframe sem quantidade de itens
df_zeroed = df_test.copy()
df_zeroed['key'] = 0
df_zeroed = pd.merge(df_zeroed,
                     pd.DataFrame({'date_block_num': np.arange(0, 34), 'key': np.zeros(34, dtype=int)}),
                     how='left', on='key').\
                set_index(['shop_id', 'item_id', 'date_block_num']).\
                drop(['key'], axis=1)
df_zeroed.head()

Empty DataFrame
Columns: []
Index: [(5, 5037, 0), (5, 5037, 1), (5, 5037, 2), (5, 5037, 3), (5, 5037, 4)]

In [8]:
df_zeroed.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7282800 entries, (5, 5037, 0) to (45, 969, 33)
Empty DataFrame

In [9]:
df_zeroed.index.names

FrozenList(['shop_id', 'item_id', 'date_block_num'])

## Sales (monthly)

In [10]:
df_monthly = pd.read_csv(\
    input_prefix + '../sales_monthly.csv' + bzfile,
    #nrows=300, # somente primeiras linhas!
    dtype={'date_block_num': np.int8, 'shop_id': np.int8, 'item_id': np.int16,
           'item_price_mean': np.int32, 'item_cnt_sum': np.int32},
    index_col=['shop_id', 'item_id', 'date_block_num']
)
print('shape:', df_monthly.shape)
df_monthly.head()

/opt/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


shape: (1608226, 2)


item_price_mean  item_cnt_sum
shop_id item_id date_block_num                               
0       30      1                           265            31
        31      1                           434            11
        32      0                           221             6
                1                           221            10
        33      0                           347             3

In [11]:
df_monthly.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1608226 entries, (0, 30, 1) to (59, 22167, 17)
Data columns (total 2 columns):
item_price_mean    1608226 non-null int32
item_cnt_sum       1608226 non-null int32
dtypes: int32(2)
memory usage: 18.6 MB


In [12]:
df_monthly.describe()

,item_price_mean,item_cnt_sum
count,1.6082e+06,1.6082e+06
mean,7.9020e+02,2.2732e+00
std,1.5493e+03,8.6532e+00
min,1.0000e+00,1.0000e+00
25%,1.9900e+02,1.0000e+00
50%,3.9900e+02,1.0000e+00
75%,8.9500e+02,2.0000e+00
max,3.0798e+05,2.2530e+03


In [13]:
df_monthly.index.names

FrozenList(['shop_id', 'item_id', 'date_block_num'])

## Zeroed joined Monthly Sales

In [14]:
# mesclar dataframes e zerar valores nulos
df_train2 = pd.merge(df_zeroed,
                     df_monthly,
                     how='left', left_index=True, right_index=True).\
                fillna(0)
df_train2.head()

item_price_mean  item_cnt_sum
shop_id item_id date_block_num                               
5       5037    0                           0.0           0.0
                1                           0.0           0.0
                2                           0.0           0.0
                3                           0.0           0.0
                4                           0.0           0.0

In [15]:
df_train2['item_price_mean'] = df_train2['item_price_mean'].astype(np.int32)
df_train2['item_cnt_sum'] = df_train2['item_cnt_sum'].astype(np.int32)

In [16]:
df_train2.head(10)

item_price_mean  item_cnt_sum
shop_id item_id date_block_num                               
5       5037    0                             0             0
                1                             0             0
                2                             0             0
                3                             0             0
                4                             0             0
                5                             0             0
                6                             0             0
                7                             0             0
                8                             0             0
                9                             0             0

In [17]:
df_train2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7282800 entries, (5, 5037, 0) to (45, 969, 33)
Data columns (total 2 columns):
item_price_mean    int32
item_cnt_sum       int32
dtypes: int32(2)
memory usage: 339.4 MB


In [18]:
df_train2.describe()

,item_price_mean,item_cnt_sum
count,7.2828e+06,7.2828e+06
mean,7.5048e+01,2.2255e-01
std,5.1399e+02,3.3251e+00
min,0.0000e+00,0.0000e+00
25%,0.0000e+00,0.0000e+00
50%,0.0000e+00,0.0000e+00
75%,0.0000e+00,0.0000e+00
max,4.2990e+04,2.2530e+03


In [19]:
df_train2.index.names

FrozenList(['shop_id', 'item_id', 'date_block_num'])

## Shops

In [20]:
df_shops = pd.read_csv(
    input_prefix + '../shops_full' + '.csv' + bzfile,
    dtype={'city_code': 'category'},
    index_col='shop_id')
print('shape:', df_shops.shape)
df_shops.head()

shape: (60, 3)


,shop_name,city_name,city_code
shop_id,,,
0,"!Якутск Орджоникидзе, 56 фран",!Якутск,1
1,"!Якутск ТЦ ""Центральный"" фран",!Якутск,1
2,"Адыгея ТЦ ""Мега""",Адыгея,2
3,"Балашиха ТРК ""Октябрь-Киномир""",Балашиха,3
4,"Волжский ТЦ ""Волга Молл""",Волжский,4


In [21]:
df_shops.drop(columns=['shop_name', 'city_name'], inplace=True)
df_shops.head()

,city_code
shop_id,
0,1
1,1
2,2
3,3
4,4


In [22]:
df_shops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 59
Data columns (total 1 columns):
city_code    60 non-null category
dtypes: category(1)
memory usage: 1.4 KB


In [23]:
df_shops.describe()

,city_code
count,60
unique,32
top,15
freq,13


## Items and Categories

In [24]:
df_items = pd.read_csv(
    input_prefix + '../items_full' + '.csv' + bzfile,
    dtype={'item_category_id': 'category', 'subject_code': 'category',
           'group_code': 'category', 'subgroup_code': 'category'},
    index_col='item_id')
print('shape:', df_items.shape)
df_items.head()

shape: (22170, 9)


,item_name,item_category_id,subject_name,subject_code,item_category_name,group_name,subgroup_name,group_code,subgroup_code
item_id,,,,,,,,,
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,40,ВО,1803,Кино - DVD,КИНО,DVD,11,8
1,!ABBYY FineReader 12 Professional Edition Full...,76,ABBYY,111,Программы - Для дома и офиса (Цифра),ПРОГРАММЫ,ДЛЯ ДОМА И ОФИСА,15,31
2,***В ЛУЧАХ СЛАВЫ (UNV) D,40,В,1614,Кино - DVD,КИНО,DVD,11,8
3,***ГОЛУБАЯ ВОЛНА (Univ) D,40,ГОЛУБАЯ,1958,Кино - DVD,КИНО,DVD,11,8
4,***КОРОБКА (СТЕКЛО) D,40,КОРОБКА,2630,Кино - DVD,КИНО,DVD,11,8


In [25]:
df_items.drop(columns=['item_name', 'subject_name', 'item_category_name', 'group_name', 'subgroup_name'], inplace=True)
df_items.head()

,item_category_id,subject_code,group_code,subgroup_code
item_id,,,,
0,40,1803,11,8
1,76,111,15,31
2,40,1614,11,8
3,40,1958,11,8
4,40,2630,11,8


In [26]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22170 entries, 0 to 22169
Data columns (total 4 columns):
item_category_id    22170 non-null category
subject_code        22170 non-null category
group_code          22170 non-null category
subgroup_code       22170 non-null category
dtypes: category(4)
memory usage: 399.6 KB


In [27]:
df_items.describe()

,item_category_id,subject_code,group_code,subgroup_code
count,22170,22170,22170,22170
unique,84,4655,18,55
top,40,4348,11,8
freq,5035,599,7464,5035


## Sales (full)

In [28]:
df_train2.head()

item_price_mean  item_cnt_sum
shop_id item_id date_block_num                               
5       5037    0                             0             0
                1                             0             0
                2                             0             0
                3                             0             0
                4                             0             0

In [29]:
df_train2.reset_index().head(10)

,shop_id,item_id,date_block_num,item_price_mean,item_cnt_sum
0,5,5037,0,0,0
1,5,5037,1,0,0
2,5,5037,2,0,0
3,5,5037,3,0,0
4,5,5037,4,0,0
5,5,5037,5,0,0
6,5,5037,6,0,0
7,5,5037,7,0,0
8,5,5037,8,0,0
9,5,5037,9,0,0


In [30]:
#shop_id	item_id	date_block_num

In [31]:
df_train3 = df_train2.join(df_shops).join(df_items)
df_train3.head()

item_price_mean  item_cnt_sum city_code  \
shop_id item_id date_block_num                                            
5       5037    0                             0             0         5   
                1                             0             0         5   
                2                             0             0         5   
                3                             0             0         5   
                4                             0             0         5   

                               item_category_id subject_code group_code  \
shop_id item_id date_block_num                                            
5       5037    0                            19          793          6   
                1                            19          793          6   
                2                            19          793          6   
                3                            19          793          6   
                4                            19          793          6   

                               subgroup_code  
shop_id item_id date_block_num                
5       5037    0                         13  
                1                         13  
                2                         13  
                3                         13  
                4                         13

In [32]:
df_train3.head(10)

item_price_mean  item_cnt_sum city_code  \
shop_id item_id date_block_num                                            
5       5037    0                             0             0         5   
                1                             0             0         5   
                2                             0             0         5   
                3                             0             0         5   
                4                             0             0         5   
                5                             0             0         5   
                6                             0             0         5   
                7                             0             0         5   
                8                             0             0         5   
                9                             0             0         5   

                               item_category_id subject_code group_code  \
shop_id item_id date_block_num                                            
5       5037    0                            19          793          6   
                1                            19          793          6   
                2                            19          793          6   
                3                            19          793          6   
                4                            19          793          6   
                5                            19          793          6   
                6                            19          793          6   
                7                            19          793          6   
                8                            19          793          6   
                9                            19          793          6   

                               subgroup_code  
shop_id item_id date_block_num                
5       5037    0                         13  
                1                         13  
                2                         13  
                3                         13  
                4                         13  
                5                         13  
                6                         13  
                7                         13  
                8                         13  
                9                         13

In [33]:
df_train3.to_csv('train_full.csv')